# Mount content from Google Drive


# Load Python files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
 
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
# Download the files based on its file ID.
# File-1
 
file_id1 = '1Z8lnKok5ufjVdBA-_f1UDmTkJoLqncqY'
module1 = drive.CreateFile({'id': file_id1})
module1.GetContentFile('datasets.py')
 
# File-2
file_id2 = '1wYVBj0dd7QcSfE4IH7Hk1rBZC562k_Ne'
module2 = drive.CreateFile({'id': file_id2})
module2.GetContentFile('deeplabv3.py')
 
# File-3
file_id3 = '1RB063J7lU1HZr8LVW55IKmw48Co_xBCO'
module3 = drive.CreateFile({'id': file_id3})
module3.GetContentFile('utils.py')
 
# File-4
file_id4 = '1_VxRsNJzUOYanJ48JgwcaWoJLtZSL85-'
module4 = drive.CreateFile({'id': file_id4})
module4.GetContentFile('resnet.py')
 
# File-5
file_id5 = '15gIBgRkR0DhdvxtTr7OMyGOiJ6W639IA'
module5 = drive.CreateFile({'id': file_id5})
module5.GetContentFile("aspp.py")
 
 
print('Python files loaded')

Mounted at /content/gdrive
Python files loaded




# Import Libraries and Files

In [ ]:
import sys
import torchvision
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2
import os
from google.colab import drive
import gc
 
os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch')
os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch/model')
os.sys.path.append('/content/gdrive/My Drive/deeplabv3/pytorch/utils')
 
import resnet
from resnet import ResNet18_OS16, ResNet34_OS16, ResNet50_OS16, ResNet101_OS16, ResNet152_OS16, ResNet18_OS8, ResNet34_OS8
from datasets import DatasetTrain, DatasetVal
from utils import add_weight_decay, MyModel
from deeplabv3 import DeepLabV3
 
 

import time
 
print('Required Libraries Imported')

Required Libraries Imported


# Initialize CUDA 

In [ ]:
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf /content/gdrive/My\ Drive/yolov3/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.5.0
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 5
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


# Set the Hyperparameters

In [ ]:
# NOTE! NOTE! change this to not overwrite all log data when you train the model:
model_id = "2"

num_epochs = 300
batch_size = 3
learning_rate = 0.0001

cuda = True if torch.cuda.is_available() else False

print('Hyperparameters set')
transform = transforms.Compose([
     
     transforms.Resize((300, 300)),
     transforms.CenterCrop((100, 100)),
     transforms.RandomCrop((80, 80)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomRotation(degrees=(-90, 90)),
     transforms.RandomVerticalFlip(p=0.5),
     transforms.Resize((512,1024)),
     transforms.ToTensor()
     #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ])

Hyperparameters set


#Upload the pretrained ResNet18 model

In [ ]:
#Note: only import if it is already not imported
from google.colab import files
#uploaded = files.upload()
!ls

adc.json  datasets.py	gdrive	     resnet.py	  utils.py
aspp.py   deeplabv3.py	__pycache__  sample_data


# Define the Model

In [ ]:
proj_dir = "/content/gdrive/My Drive/deeplabv3/pytorch/"

network = DeepLabV3(model_id, project_dir = proj_dir)

train_dataset = DatasetTrain(data_path = proj_dir + "data/train/",
                             label_path = proj_dir + "data/labels/",transforms=transform,enabletransform=False)
val_dataset = DatasetVal(data_path = proj_dir + "data/val/",
                         label_path = proj_dir + "data/labels/",transforms=transform)


num_train_batches = int(len(train_dataset)/batch_size)
num_val_batches = int(len(val_dataset)/batch_size)
print ("num_train_batches:", num_train_batches)
print ("num_val_batches:", num_val_batches)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, shuffle=True,
                                           num_workers=2, drop_last=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size, shuffle=False,
                                         num_workers=2, drop_last=True)
print(os.listdir(proj_dir+"training_logs/model_2/checkpoints")[-1])
params = add_weight_decay(network, l2_value=0.0001)
optimizer = torch.optim.Adam(params, lr=learning_rate)

#with open("/content/gdrive/My Drive/StradaImaging/deeplabv3_pytorch/data/cityscapes/meta/class_weights.pkl", "rb") as file: # (needed for python3)
#    class_weights = np.array(pickle.load(file))
#class_weights = torch.from_numpy(class_weights)
#class_weights = Variable(class_weights.type(torch.FloatTensor))#.cuda()

# loss function
loss_fn = nn.CrossEntropyLoss()#weight=class_weights)
#loss_fn = nn.MSELoss()


# pre-trained weights
device = torch.device("cuda")
#weights_path = proj_dir+"pretrained_models/model_13_2_2_2_epoch_580.pth" # start training from orignal pretrained model
weights_path = proj_dir+"training_logs/model_2/checkpoints/"+os.listdir(proj_dir+"training_logs/model_2/checkpoints")[-1] # start training from last saved weights
network.load_state_dict(torch.load(weights_path,map_location=torch.device('cuda') ),strict=False)
network.to(device)

# re define model
network = MyModel(network, model_id, proj_dir)
network.cuda()

# Define tensor types
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

pretrained resnet, 18
[{'img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/train/M1 J23a Slips_Forward_152.jpg', 'label_img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/labels/M1 J23a Slips_Forward_152.png', 'img_id': 'M1 J23a Slips_Forward_152.jpg'}, {'img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/train/M1 J23a Slips_Forward_276.jpg', 'label_img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/labels/M1 J23a Slips_Forward_276.png', 'img_id': 'M1 J23a Slips_Forward_276.jpg'}, {'img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/train/M1 J23a Slips_Forward_110.jpg', 'label_img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/labels/M1 J23a Slips_Forward_110.png', 'img_id': 'M1 J23a Slips_Forward_110.jpg'}, {'img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/train/M1 J23a Slips_Forward_2.jpg', 'label_img_path': '/content/gdrive/My Drive/deeplabv3/pytorch/data/labels/M1 J23a Slips_Forward_2.png', 'img_id': 'M1 J23a Slips_

# Training the model

In [ ]:
import os
from google.colab import drive
#drive.mount('/content/gdrive')

#os.path.exists("/content/gdrive/My Drive/StradaImaging/deeplabv3_pytorch/data/cityscapes/meta")
os.path.exists("/content/gdrive/My Drive/deeplabv3")


True

In [ ]:
  start_time = time.time()
epoch_losses_train = []
epoch_losses_val = []
EPS=1e-6
c=0
requires_grad = True
iou_train = []
iou_val = []
for epoch in range(num_epochs):
    print ("###########################")
    print ("######## NEW EPOCH ########")
    print ("###########################")
    print ("epoch: %d/%d" % (epoch+1, num_epochs))

    current_time = time.time()
    print (epoch, " ", (time.time() - start_time)/60, "minutes  taken for the previous epoch")
    
    ############################################################################
    # train:
    ############################################################################
    network.train() # (set in training mode, this affects BatchNorm and dropout)
    batch_losses = []
    iou_train = []
    #train_acc = []
    for step, (imgs, label_imgs) in enumerate(train_loader):
        current_time = time.time()
        print (step, "/", num_train_batches, " ", (time.time() - start_time)/60, "minutes  since training start")
        print("batch# ", step)
        #imshow(torchvision.utils.make_grid(imgs))
        #torch.cuda.empty_cache()
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        #imgs.requires_grad = True
        label_imgs = Variable(label_imgs.type(torch.LongTensor)).cuda()# (shape: (batch_size, img_h, img_w))
        
     
        gc.collect()
        torch.cuda.empty_cache()
        #print(imgs.shape)
        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))
        #print(outputs.shape)
        
        # compute the loss:
        #outputs=outputs.cpu().detach().numpy()
        #outputs=outputs[:,:+1,:,:]
        #outputs=outputs.reshape(3,256,256)
        loss = loss_fn(outputs , label_imgs)
        #loss = loss_fn(outputs.data.squeeze(1), label_imgs.data.squeeze(1))
        loss_value = loss.data.cpu().numpy()
        batch_losses.append(loss_value)

        # optimization step:

        optimizer.zero_grad() # (reset gradients)
        loss.backward() # (compute gradients)
        optimizer.step() # (perform optimization step)
        
        _, predicted = torch.max(outputs.data, 1)
    #    print((predicted==1).sum(), (label_imgs==1).sum())
        correct = float((predicted == label_imgs).sum().item())
        total = float(batch_size*label_imgs.size(1)*label_imgs.size(2))    
      
        #accu = (100 * correct / total)
        #train_acc.append(accu)
      
        intersection = torch.logical_and(label_imgs, predicted  )
        union = torch.logical_or(label_imgs, predicted)
     #   print(intersection.sum(), union.sum())
        iou_value = (intersection.sum() + EPS) / (union.sum() + EPS)
        iou_value = iou_value.data.cpu().numpy()

        iou_train.append(iou_value)

        print ("%d/%d: %.6f sec, train_loss: %f, iou: %f" % (step+1, num_train_batches, (time.time() - start_time), loss_value, iou_value))

    #train_iou = 100*np.mean(iou_train) 
    #print('train iou: %f %%' % train_iou)

    #epochtrain_acc = np.mean(train_acc) 
    #print('train acc: %d %%' % epochtrain_acc)
        #print (time.time() - current_time)
      
    epoch_loss = np.mean(batch_losses)
    epoch_losses_train.append(epoch_loss)
    with open("%s/epoch_losses_train.pkl" % network.model_dir, "wb") as file:
        pickle.dump(epoch_losses_train, file)
    print ("train loss: %g" % epoch_loss)
    plt.figure(1)
    plt.plot(epoch_losses_train, "k^")
    plt.plot(epoch_losses_train, "k")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.title("train loss per epoch")
    plt.savefig("%s/epoch_losses_train.png" % network.model_dir)
    plt.close(1)
  
  
    #intersection = (outputs & imgs).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    #union = (outputs | imgs).float().sum((1, 2))         # Will be zzero if both are 0
    
    #iou = (intersection + SMOOTH) / (union + SMOOTH)
    #print ("iou="+iou)
    print ("####")

    ############################################################################
    # val:
    ############################################################################
    network.eval() # (set in evaluation mode, this affects BatchNorm and dropout)
    batch_losses = []
    val_acc = []
    iou_valid = []
    val_ap = []
    TP = 0
    FP = 0
    
    for step, (imgs, label_imgs, img_ids) in enumerate(val_loader):
        #with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        for var_i in range(1):
            #plt.imshow(imgs)
            imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
            label_imgs = Variable(label_imgs.type(torch.LongTensor)).cuda() # (shape: (batch_size, img_h, img_w))
            
          
            # compute the loss:
            outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))

            loss = loss_fn(outputs , label_imgs)
            loss_value = loss.data.cpu().numpy()
            batch_losses.append(loss_value)
            _, predicted = torch.max(outputs.data, 1)
            
            correct = float((predicted == label_imgs).sum().item())
            total = float(batch_size*label_imgs.size(1)*label_imgs.size(2))    
            outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
            pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
            pred_label_imgs = pred_label_imgs.astype(np.uint8)

            for i in range(pred_label_imgs.shape[0]):
                pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
                img_id = img_ids[i]
                img = imgs[i] # (shape: (3, img_h, img_w))

                img = img.data.cpu().numpy()
                img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
                img = img*np.array([0.229, 0.224, 0.225])
                img = img + np.array([0.485, 0.456, 0.406])
                img = img*255.0
                img = img.astype(np.uint8)

                pred_label_img_color = label_img_to_color(pred_label_img)
                overlayed_img = 0.35*img + 0.65*pred_label_img_color
                overlayed_img = overlayed_img.astype(np.uint8)

                img_h = overlayed_img.shape[0]
                img_w = overlayed_img.shape[1]

            # TODO! do this using network.model_dir instead
              
                cv2.imwrite(network.model_dir + "/" + img_id + "_overlayed.png", overlayed_img)
            #accu = (100 * correct / total)
            val_acc.append(accu)
            intersection = torch.logical_and(label_imgs, predicted)
            union = torch.logical_or(label_imgs, predicted)
            iou_value = (intersection.sum() + EPS) / (union.sum() + EPS)
            iou_value = iou_value.data.cpu().numpy()

            iou_val.append(iou_value)

                
            if iou_value>0.5:
                TP += 1
                #   print("TP")
            else: 
                FP += 1
             #   print("FP")
            # print('iou: %f' % iou_value)
            iou_valid.append(iou_value)
            if (TP + FP):
              AP = TP / (TP + FP)
            else:
              AP = 0

            val_ap.append(AP)
              #  print('AP: %f' % AP)


            #    dice = dice_metric(predicted, label_imgs)
          

            print ("%d/%d: %.6f sec, val_loss: %f, iou: %f" % (step+1, num_val_batches, (time.time() - start_time), loss_value, iou_value))
      

    mAP = 100*np.mean(val_ap)
    print('mAP: %f' % mAP)        

    mean_iou = np.mean(iou_val) 
    print('val iou: %f %%' % mean_iou)
    epoch_loss = np.mean(batch_losses)
    epoch_losses_val.append(epoch_loss)
    with open("%s/epoch_losses_val.pkl" % network.model_dir, "wb") as file:
        pickle.dump(epoch_losses_val, file)
    print ("val loss: %g" % epoch_loss)
    plt.figure(1)
    plt.plot(epoch_losses_val, "k^")
    plt.plot(epoch_losses_val, "k")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.title("val loss per epoch")
    plt.savefig("%s/epoch_losses_val.png" % network.model_dir)
    plt.close(1)
    c+=1
    # save the model weights to disk:
    if (c==10):
      c=0
      print("10 epochs completed saving weights")
      checkpoint_path = network.checkpoints_dir + "/model_" + model_id +"_epoch_" + str(epoch+1) + ".pth"
      torch.save(network.state_dict(), checkpoint_path)


###########################
######## NEW EPOCH ########
###########################
epoch: 1/300
0   4.746913909912109e-05 minutes  taken for the previous epoch
0 / 81   0.015435171127319337 minutes  since training start
batch#  0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


1/81: 3.456013 sec, train_loss: 0.836160, iou: 0.000000
1 / 81   0.057642428080240886 minutes  since training start
batch#  1
2/81: 5.637800 sec, train_loss: 0.856760, iou: 0.000075
2 / 81   0.09407593806584676 minutes  since training start
batch#  2
3/81: 7.842275 sec, train_loss: 0.911882, iou: 0.000000
3 / 81   0.13081332445144653 minutes  since training start
batch#  3
4/81: 10.012245 sec, train_loss: 0.809099, iou: 0.000000
4 / 81   0.16697625716527303 minutes  since training start
batch#  4
5/81: 12.226005 sec, train_loss: 0.808973, iou: 0.000000
5 / 81   0.2038564165433248 minutes  since training start
batch#  5
6/81: 14.428952 sec, train_loss: 0.830498, iou: 0.000000
6 / 81   0.24059254328409832 minutes  since training start
batch#  6
7/81: 16.626537 sec, train_loss: 0.813350, iou: 0.000000
7 / 81   0.2772005240122477 minutes  since training start
batch#  7
8/81: 18.816687 sec, train_loss: 0.979843, iou: 0.005465
8 / 81   0.3137189944585164 minutes  since training start
batch# 

NameError: ignored

In [ ]:
import torch

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Create dummy target image
nb_classes = 19 - 1 # 18 classes + background
idx = np.linspace(0., 1., nb_classes)
cmap = matplotlib.cm.get_cmap('viridis')
rgb = cmap(idx, bytes=True)[:, :3]  # Remove alpha value

h, w = 190, 100
rgb = rgb.repeat(1000, 0)
target = np.zeros((h*w, 3), dtype=np.uint8)
target[:rgb.shape[0]] = rgb
target = target.reshape(h, w, 3)

plt.imshow(target) # Each class in 10 rows

# Create mapping
# Get color codes for dataset (maybe you would have to use more than a single
# image, if it doesn't contain all classes)
target = torch.from_numpy(target)
colors = torch.unique(target.view(-1, target.size(2)), dim=0).numpy()
print(colors.shape)
target = target.permute(2, 0, 1).contiguous()

mapping = {tuple(c): t for c, t in zip(colors.tolist(), range(len(colors)))}

mask = torch.empty(h, w, dtype=torch.long)
for k in mapping:
    # Get all indices for current class
    idx = (target==torch.tensor(k, dtype=torch.uint8).unsqueeze(1).unsqueeze(2))
    validx = (idx.sum(0) == 3)  # Check that all channels match
    mask[validx] = torch.tensor(mapping[k], dtype=torch.long)

In [ ]:
imgs.requires_grad

In [ ]:
!ls
dir(imgs)
imgs.__sizeof__()

In [ ]:
#Note: only import if it is already not imported
from google.colab import files

uploaded = files.upload()

In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread("M1 J23a Slips_Forward_271.png") 

plt.imshow(img)

In [ ]:
import PIL
red_image = PIL.Image.open("M1 J23a Slips_Forward_271.png")



red_image_rgb = red_image.convert("RGB")



rgb_pixel_value = red_image_rgb.getpixel((2650,900))


print(rgb_pixel_value)

In [41]:
a=[8,4,144,3,3,21]
print(len(a))
m=0
n=0
max=a[0]
for i in range(0,len(a)-1):
  n=a[i]
  if max<n:
    max=n
    imax=i
print (max,imax)
min=a[0]
for i in range(0,len(a)-1):
  n=a[i]
  if min>n:
    min=n
    imin=i
print (min,imin)
half=(max-min)/2
for i in range(0,len(a)-1):
  

  

6
144 2
3 3
